In [1]:
# 載入資料

import pandas as pd

train_df = pd.read_csv("house/train.csv", encoding="UTF8")
test_df = pd.read_csv("house/test.csv", encoding="UTF8")

# train_df
# 1460 rows × 81 columns
# test_df
# 1459 rows × 80 columns

In [2]:
# 畫 pd 圖
# 把 train 和 test 和在一起，一起補值。

# pd.concat = 把兩個 df 合在一起。
train_df_drop = train_df.drop("SalePrice", axis = 1)

datas = pd.concat([train_df_drop, test_df], axis=0)
datas = datas.drop(['Id'], axis=1)

# datas
# 2919 rows × 80 columns
# 1460 + 1459 = 2919

In [3]:
# 找空值
na = datas.isna().sum()
# na

# 要找 na > 0 的，兼排序。
na2 = na[na != 0 ].sort_values(ascending=False)
# na2

In [4]:
# 缺太多直接 drop 掉
# na1000 = na[na > 1000].index

datas = datas.drop(na[na > 1000].index, axis=1)

# datas
# 2919 rows × 74 columns

In [5]:
# 這次只填數值的缺失
# 類別缺失 直接 > one-hot encoding
med = datas.median().drop("MSSubClass")
datas = datas.fillna(med)

# datas
# 2919 rows × 74 columns

In [6]:
# one-hot encoding
# pandas.get_dummies() = 函數功能：將類別變量轉換為標籤變量。
datas = pd.get_dummies(datas)
# 2919 rows × 270 columns

# 特別處理 "MSSubClass"
# "MSSubClass" 這時還有缺失值
datas = pd.get_dummies(datas, columns=["MSSubClass"])
# datas
# 2919 rows × 285 columns

In [7]:
# 資料預處理 --->

# train_df.shape
# (1460, 81)

# 根据序列取 橫列 > iloc
# datas 是 兩張表 concat 來的，所以這個時候要拆回去。
x_train = datas.iloc[ : train_df.shape[0]]
x_predict = datas.iloc[train_df.shape[0] : ]

# 答案
y_train = train_df["SalePrice"]

# 預備
predict_id = test_df["Id"]

In [8]:
# 作弊
# 不算預測力喔
# 預測力要參考鐵達尼號喔

# '真樹' 在此
# 來預測喔--->

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

clf = RandomForestRegressor(n_estimators = 75, max_depth = 8)
clf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=75, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [9]:
# 來預言喔

pre = clf.predict(x_predict)
# pre

In [10]:
# 預言要補上對應 ID

result = pd.DataFrame({
        "Id":predict_id,
        "SalePrice":pre,})

# 存檔
result.to_csv("house_price_df.csv", encoding="UTF8", index=False)

# result